In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5186090%2F8656737%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240610%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240610T170412Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0a2c6f0c75f6f858887092dbfc61e5091131ff38ec742e2cea31d4ac4e71745f1504657c2fd39aca193ad3da81c5d4ce1dbef8b33860e0438a7af6632a1241859d02b15c49c6c3b23c817a9ea4a10106b8c08b637527e37c0f40c0eb56715f7076e82a18b3653d80a290ca7d516411dfd9f41b1538dd7409ab6f65c5574264b4f40cd920a18b87663ffc640c71ce2843e4c9615613ece4458f058b3b834a3d6055eedb899cac69820c4b30285d92e394d644dd06ba0174396a406323f209428327420b6c8cb2100baf6064af4ca12af2d0c7ab8cfc70ca50e2581ff1d82deb47ae7088efc5818ad0a050c0f4fa68a770e60d1106c4f5e80e7bcefa67f4554705,vnfddataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5186718%2F8657625%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240610%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240610T170412Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e1ead7ea9661b7595e505937db8793317db1b5fbe5e5683304e97a57c19e4324bbf24929bc68352db9bdda1340225d547dd54983d25799d06ca5dffa8efc01bd919a0716f4d1a9fd1b679217fdce136c3d89bebfa8b9cc0b4aaeb04696a8f86da487a2b20f9c58f881812a966855cc3aa36c424bcc80b14e2ac5bfd5456a4b431572915bc95f0dc4bf2d5cbc04c29174cd8f3879e926ad29e5bc4c37b217079e8096c2d3f3c8e4e6381db5d612edf06a90e9ddae2ebe004d870ac84549b4249f18b9d50c146b3280c3d3c4d743d72509a7668befacbb0995c3bc162ace01384aac5232228e90a0d03c54930b4a601cfa750f30c3aced3c5facc9f28d55da756,stopwords:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5186832%2F8657777%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240610%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240610T170412Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6c00187e4d93afc5da1a72385dbbb3ac2a81db7ba6f38f65d1e35de3bb8bc80fae1d4c9392c892f265aae7bc1151cf2ecf2ffd0be8aef8db02e6a5c35ef1fddc706a35cbb3901d025488432c0e9c848efec12a1e09cd8423e1a70226661b13ebcdbafe921484ef9541e58888314f80be26ea062412cad5382118e5f8ab9509a4fbd3ebf62ad6b3ff16af1922656d847d6aee9a5b2c87f4edc11b17dbc45f5904c99e3213ddda1a285fa4446003da3e2413f0f822a72e095dc28bd2554ffc2b41bb925093ef7c082757c41b1879e8deb5dad0ded9c514acaf397941ee024819733de9a7e2e8cee266f5dce846028ecbf82d1e4acadaf7bfb8f807b69b7b214c13'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00:00:01


In [ ]:
# Import libraries
import pandas as pd
# Load & preprocess data
import sys
sys.path.append('Models')

# Tokenizer
from pyvi import ViTokenizer
# Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Model
from sklearn.linear_model import LogisticRegression
# Evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split



In [ ]:
data = pd.read_csv('/kaggle/input/vnfddataset/vn_news_223_tdlfr.csv')
train, test = train_test_split(data, test_size=0.3)



In [ ]:
import pandas as pd  # type: ignore
import string


def preprocess_data(
    data,
    url=True,
    punctuation=True,
    lowercase=True,
    stopword=False,

):
    # Load stopwords
    with open('/kaggle/input/stopwords/vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    # Function to remove stopwords

    def remove_stopwords(text):
        words = text.split()
        words = [word for word in words if word not in stopwords]
        return ' '.join(words)
    if url:
        # Remove URLs
        data['text'] = data['text'].str.replace(
            r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
    if punctuation:
        # Remove punctuation
        data['text'] = data['text'].str.replace(
            '['+string.punctuation+']', '', regex=True)
    if lowercase:
        # Lowercase
        data['text'] = data['text'].str.lower()
    if stopword:
        # Remove stopword
        data['text'] = data['text'].apply(remove_stopwords)

    return data

In [ ]:
train.dropna(inplace=True)
train.drop_duplicates(subset='text', inplace=True, keep='first')

In [ ]:
train = preprocess_data(train)

In [ ]:
train['text'].dropna(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import svm

In [ ]:

train['text'] = train['text'].astype(str).apply(ViTokenizer.tokenize)
X_train = train['text']
y_train = train['label']

In [ ]:
# vectorizer = TfidfVectorizer()
# X_train = X_train.apply(' '.join)
# X_train = vectorizer.fit_transform(X_train)

train['text'] = train['text'].astype(str).apply(ViTokenizer.tokenize)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['text'])
y_train = train['label']

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}
# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=30)

# Create a GridSearchCV object

grid = GridSearchCV(svm.SVC(), param_grid, cv=stratified_kfold, refit=True, verbose=3, n_jobs=-1)
# Fit the model to the data
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)

Fitting 30 folds for each of 100 candidates, totalling 3000 fits
[CV 4/30] END .......C=0.1, gamma=1, kernel=rbf;, score=0.500 total time=   0.1s
[CV 8/30] END .......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 12/30] END ......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 16/30] END ......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 20/30] END ......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 23/30] END ......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 27/30] END ......C=0.1, gamma=1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 1/30] END .....C=0.1, gamma=0.1, kernel=rbf;, score=0.667 total time=   0.1s
[CV 5/30] END .....C=0.1, gamma=0.1, kernel=rbf;, score=0.500 total time=   0.1s
[CV 11/30] END ....C=0.1, gamma=0.1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 12/30] END ....C=0.1, gamma=0.1, kernel=rbf;, score=0.600 total time=   0.1s
[CV 19/30] END ....C=0.1, gamma=0.1, kernel=

In [ ]:
clf = grid.best_estimator_

In [ ]:
# Training performance
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00        65

    accuracy                           1.00       156
   macro avg       1.00      1.00      1.00       156
weighted avg       1.00      1.00      1.00       156



In [ ]:
# Evaluate the model on the test set
test = preprocess_data(test, url=True, punctuation=True, stopword=False)
X_test = vectorizer.transform(test['text'])
y_test = test['label']
# Make predictions on the test set
y_test_pred = clf.predict(X_test)
# Compute accuracy
print('Accuracy on the test set:', accuracy_score(y_test, y_test_pred))
# Compute F1 score
print(classification_report(y_test, y_test_pred))

Accuracy on the test set: 0.8955223880597015
              precision    recall  f1-score   support

           0       0.86      0.94      0.90        32
           1       0.94      0.86      0.90        35

    accuracy                           0.90        67
   macro avg       0.90      0.90      0.90        67
weighted avg       0.90      0.90      0.90        67

